<a href="https://colab.research.google.com/github/kongminhyuk/swboot_camp/blob/main/base_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1. Install
warning은 무시해도 됩니다
임의로 버전을 고정한거라 뜨는 메세지 같아요

In [ ]:
!pip install --upgrade setuptools
!pip install fastapi kaleido python-multipart uvicorn
#!pip install jedi
!pip install transformers==4.16.2
!pip install sentencepiece
!pip install datasets
!pip install rouge_score
!pip install pytorch_lightning==1.5.10

  Using cached setuptools-68.2.2-py3-none-any.whl (807 kB)
  Attempting uninstall: setuptools
    Found existing installation: setuptools 59.5.0
    Uninstalling setuptools-59.5.0:
      Successfully uninstalled setuptools-59.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.
pytorch-lightning 1.5.10 requires setuptools==59.5.0, but you have setuptools 68.2.2 which is incompatible.


  Using cached setuptools-59.5.0-py3-none-any.whl (952 kB)
  Attempting uninstall: setuptools
    Found existing installation: setuptools 68.2.2
    Uninstalling setuptools-68.2.2:
      Successfully uninstalled setuptools-68.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.
arviz 0.15.1 requires setuptools>=60.0.0, but you have setuptools 59.5.0 which is incompatible.
cvxpy 1.3.2 requires setuptools>65.5.1, but you have setuptools 59.5.0 which is incompatible.


In [ ]:
!git clone https://github.com/lbox-kr/lbox_open.git --branch v0.1
%cd lbox_open

Cloning into 'lbox_open'...
remote: Enumerating objects: 266, done.
remote: Counting objects: 100% (266/266), done.
remote: Compressing objects: 100% (161/161), done.
remote: Total 266 (delta 128), reused 210 (delta 94), pack-reused 0
Receiving objects: 100% (266/266), 86.64 KiB | 8.66 MiB/s, done.
Resolving deltas: 100% (128/128), done.
Note: switching to '5dd718fe2424d84bc73a78890b357b429c2f9cb0'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false

/content/lbox_open/lbox_open


In [ ]:
import os
from argparse import Namespace

import torch
import transformers
import pytorch_lightning as pl

from lcube.data_module.data_lbox_open import LBoxOpenDataModule
from lcube.model.model_baseline import SeqToSeqBaseline

device = 'cuda' if torch.cuda.is_available() else "cpu"

#2.Parameter 생성

In [ ]:
args = Namespace()
# dataset
args.dataset_card = "lbox/lbox_open"
args.task = "casename_classification"  # comment and uncomment following lines depending on the task
# args.task = "statute_classification"
# args.task = "summarization"


if args.task in ["casename_classification", "statute_classification"]:
    args.input_key = "facts"

    # model
    args.model_card = "google/mt5-small"
    args.max_input_len = 512
    args.max_target_len = 64

    # train
    args.max_epochs = 10
    args.learning_rate = 2e-4
    args.batch_size = 8
    args.batch_size_eval = 2 * args.batch_size
    args.accumulate_grad_batches = 1
    args.validation_metric = "exact_match"

elif args.task == "summarization":
    args.input_key = "precedent"

    # model
    args.model_card = "google/mt5-small"
    args.max_input_len = 768
    args.max_target_len = 512

    # train
    args.max_epochs = 10
    args.learning_rate = 2e-4
    args.batch_size = 1
    args.batch_size_eval = 2 * args.batch_size
    args.accumulate_grad_batches = 8
    args.validation_metric = "rougeL"

else:
    raise ValueError


args.tokenizer = transformers.MT5TokenizerFast.from_pretrained(args.model_card)
pl.seed_everything(seed=1, workers=False)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
INFO:pytorch_lightning.utilities.seed:Global seed set to 1


1

# 3. Datasets 불러오기

In [ ]:
data_module = LBoxOpenDataModule(
    args.dataset_card,
    args.task,
    args.tokenizer,
    args.max_input_len,
    args.max_target_len,
    args.batch_size,
    args.batch_size_eval,
)

# 4. 모델 준비
MT5ForConditionalGeneration

In [ ]:
backbone = transformers.MT5ForConditionalGeneration.from_pretrained(args.model_card)
model = SeqToSeqBaseline(
    args.task,
    backbone,
    args.tokenizer,
    args.learning_rate,
    args.max_target_len,
    args.validation_metric

)

# 5. 학습 준비
epoch 5~10 정도 해야할듯 합니다




In [ ]:
# PyTorch Lightning을 사용하여 모델을 훈련하고 지정된
# 검증 메트릭에 대한 최고 성능을 갖는 모델을 저장하기 위해
# ModelCheckpoint 콜백을 설정합니다.
callbacks = pl.callbacks.ModelCheckpoint(
    #monitor=args.validation_metric,  # 검증 메트릭을 모니터링하여 최고 성능 모델을 찾습니다.
    monitor='val_loss',
    dirpath=f"./saved/0/{args.task}",  # 최고 성능 모델이 저장될 디렉토리입니다.
    save_top_k=1,  # 최고의 모델 하나만 저장합니다.
    mode="max",  # 모니터링 메트릭의 최대 값을 가진 모델을 저장합니다.
)

# PyTorch Lightning Trainer 인스턴스를 생성합니다.
trainer = pl.Trainer(
    max_epochs= 2,  # 최대 에폭 수는 1입니다. 더 많은 에폭으로 훈련하려면 10으로 주석 처리된 줄을 사용할 수 있습니다.
    #max_epochs = 10,
    gpus=torch.cuda.device_count(),  # 사용할 GPU 수를 설정합니다.
    accumulate_grad_batches=args.accumulate_grad_batches,  # 그래디언트 누적 배치 수를 설정합니다.
    fast_dev_run=not True,  # True로 설정하면 훈련 및 검증을 위해 하나의 배치만 실행됩니다.
    callbacks=callbacks,  # 사용할 콜백을 설정합니다.

)

INFO:pytorch_lightning.utilities.distributed:GPU available: True, used: True
INFO:pytorch_lightning.utilities.distributed:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.distributed:IPU available: False, using: 0 IPUs


# 6. 학습
평균 1회당 9분 정도 소요

In [ ]:
trainer.fit(model, data_module)

INFO:pytorch_lightning.accelerators.gpu:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type                        | Params
------------------------------------------------------
0 | model | MT5ForConditionalGeneration | 300 M 
------------------------------------------------------
300 M     Trainable params
0         Non-trainable params
300 M     Total params
1,200.707 Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.seed:Global seed set to 1


metric: exact_match, score: 0.0
Validation test
ground truth: 감염병의예방및관리에관한법률위반
prediction:   <extra_id_0>하였다.


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

metric: exact_match, score: 0.0
Validation test
ground truth: 감염병의예방및관리에관한법률위반
prediction:   


MisconfigurationException: ignored

In [ ]:
class YourLightningModule(pl.LightningModule):
    def training_step(self, batch, batch_idx):
        # 훈련 로직을 여기에 추가
        loss = ...  # 훈련 손실 계산

        # 훈련 손실을 로그에 기록합니다
        self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        # 검증 로직을 여기에 추가
        loss = ...  # 검증 손실 계산

        # 검증 손실을 로그에 기록합니다
        self.log('val_loss', loss, on_epoch=True, prog_bar=True)
        return loss

In [ ]:
import matplotlib.pyplot as plt

# 저장된 모델 체크포인트의 경로를 설정합니다.
model_checkpoint_path = "./saved_model.ckpt"

# trainer.callback_metrics에서 훈련 및 검증 손실에 접근합니다.
train_losses = trainer.callback_metrics['train_loss']
val_losses = trainer.callback_metrics['val_loss']

# 플로팅
plt.plot(train_losses, label='Train Loss')
plt.plot(val_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

KeyError: ignored

# 7. Test
테스트는 input이 고정이여서 정확도 체크 용도로 쓰면 될듯합니다

In [ ]:
input_text = data_module.dataset["test"][40][args.input_key]

model_inputs = args.tokenizer(
            input_text,
            max_length=args.max_input_len,
            padding=True,
            truncation=True,
            return_tensors='pt',
        )
model_inputs = {k: v.to(device) for k,v in model_inputs.items()}

In [ ]:
model.model = model.model.to(device)
pr_seqs = model.model.generate(model_inputs["input_ids"], max_length=args.max_target_len)
prs = args.tokenizer.batch_decode(pr_seqs, skip_special_tokens=True)
print(f"Input\n {input_text}\n\n")
print(f"Prediction\n {prs}")

Input
 검색하실 판례를 입력해 주세요:


Prediction
 ['<extra_id_0>)무추서']


#8. input을 임의로 넣을 수 있는 코드 추가

In [ ]:
# 사용자가 직접 입력한 텍스트를 사용
user_input_text = "검색할 판례를 작성해주세요:"

# 모델 입력을 설정
model_inputs = args.tokenizer(

    user_input_text,
    max_length=args.max_input_len,
    padding=True,
    truncation=True,
    return_tensors='pt',
)
model_inputs = {k: v.to(device) for k, v in model_inputs.items()}

# 모델에 대한 예측 생성
model.model = model.model.to(device)
pr_seqs = model.model.generate(model_inputs["input_ids"], max_length=args.max_target_len)

# 디코딩하여 출력
prs = args.tokenizer.batch_decode(pr_seqs, skip_special_tokens=True)

# 입력 및 예측 출력
print(f"Input\n {user_input_text}\n\n")
print(f"Prediction\n {prs}")

Input
 검색할 판례를 작성해주세요:


Prediction
 ['손해등이용nissen신청']
